In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
CLIENT_ID = 'NIEATU4RVKGO51BVVGICAAZA1OROT0JFKUDIBBHY1Y11MCFV' # your Foursquare ID
CLIENT_SECRET = 'SMWBZRGEEWM3YFDV2TVCBCSL44J13ZSYCTD233WIXTUHYAKG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NIEATU4RVKGO51BVVGICAAZA1OROT0JFKUDIBBHY1Y11MCFV
CLIENT_SECRET:SMWBZRGEEWM3YFDV2TVCBCSL44J13ZSYCTD233WIXTUHYAKG


In [4]:
address = 'Disneyland Anaheim, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
CA_latitude = location.latitude
CA_longitude = location.longitude
print(CA_latitude, CA_longitude)

33.813647599999996 -117.91971592742885


In [5]:
address2 ='Walt Disney World, Florida'
location = geolocator.geocode(address2)
FL_latitude = location.latitude
FL_longitude = location.longitude
print(FL_latitude, FL_longitude)

28.4057598 -81.5834365


In [6]:
#Search query used to create link to fetch data about hotels near disneyland florida and california
search_query = 'Hotel'
#50000 = 50KM
radius = 50000
print(search_query + ' .... OK!')


Hotel .... OK!


In [7]:
#Links to hotels around disney land california
url_CA_Hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, CA_latitude, CA_longitude, VERSION, search_query, radius, LIMIT)
url_FL_Hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, FL_latitude, FL_longitude, VERSION, search_query, radius, LIMIT)

# Hotel Data Near disneyland California

In [8]:
#fetch result for hotels in california
results_CA = requests.get(url_CA_Hotel).json()
Hotel_CA = results_CA['response']['venues']
# tranform venues into a dataframe CA
dataframe_CA = pd.json_normalize(Hotel_CA)
dataframe_CA.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,47f69cabf964a520b44e1fe3,Disneyland Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592678273,False,1150 W Magic Way,at Disneyland Dr,33.808703,-117.926752,"[{'label': 'display', 'lat': 33.8087026, 'lng'...",852,92802,US,Anaheim,CA,United States,"[1150 W Magic Way (at Disneyland Dr), Anaheim,...",NaN,NaN
1,4ae286cbf964a520de8e21e3,Howard Johnson Anaheim Hotel and Water Playground,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592678273,False,1380 S Harbor Blvd,NaN,33.812999,-117.913949,"[{'label': 'display', 'lat': 33.81299914915775...",538,92802,US,Anaheim,CA,United States,"[1380 S Harbor Blvd, Anaheim, CA 92802, United...",139099383,NaN
2,4bfec5b0369476b0ead58c1f,American Tavern (at Hotel Ménage),"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",v-1592678273,False,1221 S Harbor Blvd,at Ball Rd,33.817530,-117.915891,"[{'label': 'display', 'lat': 33.8175301898457,...",558,92805,US,Anaheim,CA,United States,"[1221 S Harbor Blvd (at Ball Rd), Anaheim, CA ...",NaN,NaN
3,4eaf7e3149010398c89ab28e,Poolside Bar at Hotel Menage,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",v-1592678273,False,NaN,NaN,33.817353,-117.916148,"[{'label': 'display', 'lat': 33.81735254352598...",528,NaN,US,NaN,California,United States,"[California, United States]",NaN,NaN
4,4a1764d8f964a52055791fe3,Clarion Hotel Anaheim Resort,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592678273,False,616 W Convention Way,at S Harbor Blvd,33.799447,-117.916133,"[{'label': 'display', 'lat': 33.79944734800926...",1615,92802,US,Anaheim,CA,United States,"[616 W Convention Way (at S Harbor Blvd), Anah...",NaN,NaN


In [9]:
#cleaning CA DATA
filtered_columns = ['name', 'categories'] + [col for col in dataframe_CA.columns if col.startswith('location.')] + ['id']
dataframe_filtered_CA = dataframe_CA.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_CA['categories'] = dataframe_filtered_CA.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_CA.columns = [column.split('.')[-1] for column in dataframe_filtered_CA.columns]

X = dataframe_filtered_CA[dataframe_filtered_CA['categories']=='Hotel']
X.reset_index(drop=True, inplace=True)
X

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Disneyland Hotel,Hotel,1150 W Magic Way,at Disneyland Dr,33.808703,-117.926752,"[{'label': 'display', 'lat': 33.8087026, 'lng'...",852,92802,US,Anaheim,CA,United States,"[1150 W Magic Way (at Disneyland Dr), Anaheim,...",NaN,47f69cabf964a520b44e1fe3
1,Howard Johnson Anaheim Hotel and Water Playground,Hotel,1380 S Harbor Blvd,NaN,33.812999,-117.913949,"[{'label': 'display', 'lat': 33.81299914915775...",538,92802,US,Anaheim,CA,United States,"[1380 S Harbor Blvd, Anaheim, CA 92802, United...",NaN,4ae286cbf964a520de8e21e3
2,Clarion Hotel Anaheim Resort,Hotel,616 W Convention Way,at S Harbor Blvd,33.799447,-117.916133,"[{'label': 'display', 'lat': 33.79944734800926...",1615,92802,US,Anaheim,CA,United States,"[616 W Convention Way (at S Harbor Blvd), Anah...",NaN,4a1764d8f964a52055791fe3
3,Disney's Grand Californian Hotel & Spa,Hotel,1600 S Disneyland Dr,at Disneyland Resort,33.807628,-117.922956,"[{'label': 'display', 'lat': 33.8076278, 'lng'...",734,92802,US,Anaheim,CA,United States,"[1600 S Disneyland Dr (at Disneyland Resort), ...",NaN,45db8e2df964a520fc421fe3
4,Holiday Inn Hotel & Suites Anaheim (1 Blk/Disn...,Hotel,1240 S Walnut St,NaN,33.816652,-117.927933,"[{'label': 'display', 'lat': 33.81665213318463...",830,92802,US,Anaheim,CA,United States,"[1240 S Walnut St, Anaheim, CA 92802, United S...",The Anaheim Resort,4b53b0cff964a520f9a727e3
5,Hotel Indigo Anaheim,Hotel,435 W Katella Ave,NaN,33.803529,-117.913588,"[{'label': 'display', 'lat': 33.80352912346113...",1260,92802,US,Anaheim,CA,United States,"[435 W Katella Ave, Anaheim, CA 92802, United ...",NaN,525d6b64498e0be855ffbcdd
6,Disney's Paradise Pier Hotel,Hotel,1717 S Disneyland Dr,at Disneyland Resort,33.806157,-117.924676,"[{'label': 'display', 'lat': 33.8061566, 'lng'...",951,92802,US,Anaheim,CA,United States,"[1717 S Disneyland Dr (at Disneyland Resort), ...",NaN,4aad8a6ef964a520cc6020e3
7,Sheraton Park Hotel at the Anaheim Resort,Hotel,1855 S Harbor Blvd,NaN,33.801186,-117.915902,"[{'label': 'display', 'lat': 33.8011859, 'lng'...",1431,92802,US,Anaheim,CA,United States,"[1855 S Harbor Blvd, Anaheim, CA 92802, United...",NaN,4cbc551e7a5d9eb0135c31e9
8,Anaheim Plaza Hotel & Suites,Hotel,1700 S Harbor Blvd,at Disney Way.,33.806128,-117.914490,"[{'label': 'display', 'lat': 33.806128, 'lng':...",966,92802,US,Anaheim,CA,United States,"[1700 S Harbor Blvd (at Disney Way.), Anaheim,...",NaN,4ad73282f964a520f70821e3
9,"SunCoast Park Hotel Anaheim, Tapestry Collecti...",Hotel,NaN,NaN,33.807700,-117.909364,"[{'label': 'display', 'lat': 33.8077, 'lng': -...",1164,92802,US,Anaheim,CA,United States,"[Anaheim, CA 92802, United States]",NaN,5deb1052df2c9e0008dd35db


In [10]:
CAid=[]
for index,row in X.iterrows():
    try:
        H_id=row['id']
        uri='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(H_id,CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(uri).json()
        Hotel_id = result['response']['venue']['rating']
        CAid.append(Hotel_id)
    except:
        CAid.append('NaN')
 
print(CAid)


[9.0, 7.8, 5.7, 8.5, 6.4, 7.2, 7.4, 6.6, 5.3, 'NaN', 6.3, 5.8, 'NaN', 'NaN', 'NaN', 5.4, 6.6]


In [11]:
X['rating'] = CAid

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
X['rating']= X['rating'].astype(str)
dropvalues = X[ X['rating'] == 'NaN' ].index
X.drop(dropvalues , inplace=True)
X.reset_index(drop=True, inplace=True)
X

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,rating
0,Disneyland Hotel,Hotel,1150 W Magic Way,at Disneyland Dr,33.808703,-117.926752,"[{'label': 'display', 'lat': 33.8087026, 'lng'...",852,92802,US,Anaheim,CA,United States,"[1150 W Magic Way (at Disneyland Dr), Anaheim,...",NaN,47f69cabf964a520b44e1fe3,9.0
1,Howard Johnson Anaheim Hotel and Water Playground,Hotel,1380 S Harbor Blvd,NaN,33.812999,-117.913949,"[{'label': 'display', 'lat': 33.81299914915775...",538,92802,US,Anaheim,CA,United States,"[1380 S Harbor Blvd, Anaheim, CA 92802, United...",NaN,4ae286cbf964a520de8e21e3,7.8
2,Clarion Hotel Anaheim Resort,Hotel,616 W Convention Way,at S Harbor Blvd,33.799447,-117.916133,"[{'label': 'display', 'lat': 33.79944734800926...",1615,92802,US,Anaheim,CA,United States,"[616 W Convention Way (at S Harbor Blvd), Anah...",NaN,4a1764d8f964a52055791fe3,5.7
3,Disney's Grand Californian Hotel & Spa,Hotel,1600 S Disneyland Dr,at Disneyland Resort,33.807628,-117.922956,"[{'label': 'display', 'lat': 33.8076278, 'lng'...",734,92802,US,Anaheim,CA,United States,"[1600 S Disneyland Dr (at Disneyland Resort), ...",NaN,45db8e2df964a520fc421fe3,8.5
4,Holiday Inn Hotel & Suites Anaheim (1 Blk/Disn...,Hotel,1240 S Walnut St,NaN,33.816652,-117.927933,"[{'label': 'display', 'lat': 33.81665213318463...",830,92802,US,Anaheim,CA,United States,"[1240 S Walnut St, Anaheim, CA 92802, United S...",The Anaheim Resort,4b53b0cff964a520f9a727e3,6.4
5,Hotel Indigo Anaheim,Hotel,435 W Katella Ave,NaN,33.803529,-117.913588,"[{'label': 'display', 'lat': 33.80352912346113...",1260,92802,US,Anaheim,CA,United States,"[435 W Katella Ave, Anaheim, CA 92802, United ...",NaN,525d6b64498e0be855ffbcdd,7.2
6,Disney's Paradise Pier Hotel,Hotel,1717 S Disneyland Dr,at Disneyland Resort,33.806157,-117.924676,"[{'label': 'display', 'lat': 33.8061566, 'lng'...",951,92802,US,Anaheim,CA,United States,"[1717 S Disneyland Dr (at Disneyland Resort), ...",NaN,4aad8a6ef964a520cc6020e3,7.4
7,Sheraton Park Hotel at the Anaheim Resort,Hotel,1855 S Harbor Blvd,NaN,33.801186,-117.915902,"[{'label': 'display', 'lat': 33.8011859, 'lng'...",1431,92802,US,Anaheim,CA,United States,"[1855 S Harbor Blvd, Anaheim, CA 92802, United...",NaN,4cbc551e7a5d9eb0135c31e9,6.6
8,Anaheim Plaza Hotel & Suites,Hotel,1700 S Harbor Blvd,at Disney Way.,33.806128,-117.914490,"[{'label': 'display', 'lat': 33.806128, 'lng':...",966,92802,US,Anaheim,CA,United States,"[1700 S Harbor Blvd (at Disney Way.), Anaheim,...",NaN,4ad73282f964a520f70821e3,5.3
9,Desert Palms Hotel and Suites,Hotel,631 W Katella Ave,NaN,33.803709,-117.916862,"[{'label': 'display', 'lat': 33.80370877516979...",1137,92802,US,Anaheim,CA,United States,"[631 W Katella Ave, Anaheim, CA 92802, United ...",NaN,4b8eaccdf964a520343033e3,6.3


In [13]:
X['rating']= X['rating'].astype(float)
dropvalues2 = X[ X['rating'] < 6 ].index
X.drop(dropvalues2 , inplace=True)
X.reset_index(drop=True, inplace=True)
X

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,rating
0,Disneyland Hotel,Hotel,1150 W Magic Way,at Disneyland Dr,33.808703,-117.926752,"[{'label': 'display', 'lat': 33.8087026, 'lng'...",852,92802,US,Anaheim,CA,United States,"[1150 W Magic Way (at Disneyland Dr), Anaheim,...",NaN,47f69cabf964a520b44e1fe3,9.0
1,Howard Johnson Anaheim Hotel and Water Playground,Hotel,1380 S Harbor Blvd,NaN,33.812999,-117.913949,"[{'label': 'display', 'lat': 33.81299914915775...",538,92802,US,Anaheim,CA,United States,"[1380 S Harbor Blvd, Anaheim, CA 92802, United...",NaN,4ae286cbf964a520de8e21e3,7.8
2,Disney's Grand Californian Hotel & Spa,Hotel,1600 S Disneyland Dr,at Disneyland Resort,33.807628,-117.922956,"[{'label': 'display', 'lat': 33.8076278, 'lng'...",734,92802,US,Anaheim,CA,United States,"[1600 S Disneyland Dr (at Disneyland Resort), ...",NaN,45db8e2df964a520fc421fe3,8.5
3,Holiday Inn Hotel & Suites Anaheim (1 Blk/Disn...,Hotel,1240 S Walnut St,NaN,33.816652,-117.927933,"[{'label': 'display', 'lat': 33.81665213318463...",830,92802,US,Anaheim,CA,United States,"[1240 S Walnut St, Anaheim, CA 92802, United S...",The Anaheim Resort,4b53b0cff964a520f9a727e3,6.4
4,Hotel Indigo Anaheim,Hotel,435 W Katella Ave,NaN,33.803529,-117.913588,"[{'label': 'display', 'lat': 33.80352912346113...",1260,92802,US,Anaheim,CA,United States,"[435 W Katella Ave, Anaheim, CA 92802, United ...",NaN,525d6b64498e0be855ffbcdd,7.2
5,Disney's Paradise Pier Hotel,Hotel,1717 S Disneyland Dr,at Disneyland Resort,33.806157,-117.924676,"[{'label': 'display', 'lat': 33.8061566, 'lng'...",951,92802,US,Anaheim,CA,United States,"[1717 S Disneyland Dr (at Disneyland Resort), ...",NaN,4aad8a6ef964a520cc6020e3,7.4
6,Sheraton Park Hotel at the Anaheim Resort,Hotel,1855 S Harbor Blvd,NaN,33.801186,-117.915902,"[{'label': 'display', 'lat': 33.8011859, 'lng'...",1431,92802,US,Anaheim,CA,United States,"[1855 S Harbor Blvd, Anaheim, CA 92802, United...",NaN,4cbc551e7a5d9eb0135c31e9,6.6
7,Desert Palms Hotel and Suites,Hotel,631 W Katella Ave,NaN,33.803709,-117.916862,"[{'label': 'display', 'lat': 33.80370877516979...",1137,92802,US,Anaheim,CA,United States,"[631 W Katella Ave, Anaheim, CA 92802, United ...",NaN,4b8eaccdf964a520343033e3,6.3
8,The Hotel Fullerton & Conference Center,Hotel,1500 S Raymond Ave,NaN,33.854993,-117.905208,"[{'label': 'display', 'lat': 33.8549926, 'lng'...",4793,92831,US,Fullerton,CA,United States,"[1500 S Raymond Ave, Fullerton, CA 92831, Unit...",NaN,50f1c531e4b033e190dd0e7b,6.6


In [14]:
#Average distance between DisneyLand resort and all above rating 6 hotels
Dis_CA_hotel=X['distance'].mean()
Dis_CA_hotel

1391.7777777777778

In [15]:
CA_map = folium.Map(location=[CA_latitude, CA_longitude], zoom_start=13) # generate map centred around the DisneyLand resort,California

# add a red circle marker to represent the DisneyLand resort,California
folium.features.CircleMarker(
    [CA_latitude, CA_longitude],
    radius=10,
    color='red',
    popup='DisneyLand resort,California',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(CA_map)

# add the Hotels as blue circle markers
for lat, lng, label in zip(X.lat, X.lng, X.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(CA_map)

# display map
CA_map

# Hotel Data Near Waltdisney Florida

In [16]:
#fetch result for hotels in california
results_FL = requests.get(url_FL_Hotel).json()
Hotel_FL = results_FL['response']['venues']
# tranform venues into a dataframe CA
dataframe_FL = pd.json_normalize(Hotel_FL)
dataframe_FL.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b05868bf964a520556522e3,Walt Disney World Dolphin Hotel,"[{'id': '4bf58dd8d48988d12f951735', 'name': 'R...",v-1592678331,False,1500 Epcot Resorts Blvd,at Buena Vista Dr,28.367503,-81.561288,"[{'label': 'display', 'lat': 28.367503, 'lng':...",4779,32830,US,Lake Buena Vista,FL,United States,"[1500 Epcot Resorts Blvd (at Buena Vista Dr), ...",103904305
1,4b548d94f964a520babf27e3,Walt Disney World Swan Hotel,"[{'id': '4bf58dd8d48988d12f951735', 'name': 'R...",v-1592678331,False,1200 Epcot Resorts Blvd,at Buena Vista Dr,28.365887,-81.558890,NaN,5047,32830,US,Lake Buena Vista,FL,United States,"[1200 Epcot Resorts Blvd (at Buena Vista Dr), ...",75725371
2,4283ee00f964a52097221fe3,Hard Rock Hotel at Universal Orlando,"[{'id': '4bf58dd8d48988d12f951735', 'name': 'R...",v-1592678331,False,5800 Universal Blvd,NaN,28.477142,-81.464524,"[{'label': 'display', 'lat': 28.47714156614553...",14093,32819,US,Orlando,FL,United States,"[5800 Universal Blvd, Orlando, FL 32819, Unite...",NaN
3,4b05868cf964a5206c6522e3,Loews Portofino Bay Hotel at Universal Orlando,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592678331,False,5601 Universal Blvd,NaN,28.480846,-81.460843,"[{'label': 'display', 'lat': 28.48084591771423...",14623,32819,US,Orlando,FL,United States,"[5601 Universal Blvd, Orlando, FL 32819, Unite...",61572298
4,4b05868af964a520266522e3,Rosen Centre Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592678331,False,9840 International Dr,NaN,28.424350,-81.464479,"[{'label': 'display', 'lat': 28.42434983796381...",11829,32819,US,Orlando,FL,United States,"[9840 International Dr, Orlando, FL 32819, Uni...",60552103


In [17]:
#cleaning FL DATA
filtered_columns = ['name', 'categories'] + [col for col in dataframe_FL.columns if col.startswith('location.')] + ['id']
dataframe_filtered_FL = dataframe_FL.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_FL['categories'] = dataframe_filtered_FL.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_FL.columns = [column.split('.')[-1] for column in dataframe_filtered_FL.columns]

Y = dataframe_filtered_FL[dataframe_filtered_FL['categories']=='Hotel']
Y.reset_index(drop=True, inplace=True)
Y

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Loews Portofino Bay Hotel at Universal Orlando,Hotel,5601 Universal Blvd,NaN,28.480846,-81.460843,"[{'label': 'display', 'lat': 28.48084591771423...",14623,32819,US,Orlando,FL,United States,"[5601 Universal Blvd, Orlando, FL 32819, Unite...",4b05868cf964a5206c6522e3
1,Rosen Centre Hotel,Hotel,9840 International Dr,NaN,28.424350,-81.464479,"[{'label': 'display', 'lat': 28.42434983796381...",11829,32819,US,Orlando,FL,United States,"[9840 International Dr, Orlando, FL 32819, Uni...",4b05868af964a520266522e3
2,"Grand Bohemian Hotel Orlando, Autograph Collec...",Hotel,325 South Orange Avenue,NaN,28.538690,-81.378645,"[{'label': 'display', 'lat': 28.5386903, 'lng'...",24911,32801,US,Orlando,FL,United States,"[325 South Orange Avenue, Orlando, FL 32801, U...",4b05868bf964a520546522e3
3,Seralago Hotel & Suites Main Gate East,Hotel,5678 W Irlo Bronson Memorial Hwy,Seralago Blvd,28.331714,-81.511398,"[{'label': 'display', 'lat': 28.3317138, 'lng'...",10850,34746,US,Kissimmee,FL,United States,[5678 W Irlo Bronson Memorial Hwy (Seralago Bl...,4b05868bf964a5205c6522e3
4,"Castle Hotel, Autograph Collection",Hotel,8629 International Drive,NaN,28.440175,-81.469128,"[{'label': 'display', 'lat': 28.4401754, 'lng'...",11828,32819,US,Orlando,FL,United States,"[8629 International Drive, Orlando, FL 32819, ...",5009edfde4b0b84fa25c768e
5,Melia Orlando Suite Hotel at Celebration,Hotel,225 Celebration Pl,NaN,28.331508,-81.537966,"[{'label': 'display', 'lat': 28.33150816213022...",9389,34747,US,Celebration,FL,United States,"[225 Celebration Pl, Celebration, FL 34747, Un...",54c11ca6498ee52a436a6d4f
6,Homerun Hotel Buildings,Hotel,All Star Sports Resort,Walt Disney World,28.341798,-81.574689,"[{'label': 'display', 'lat': 28.34179838953522...",7171,32830,US,Lake Buena Vista,FL,United States,"[All Star Sports Resort (Walt Disney World), L...",4ee155d330f87dc0a5df97bf
7,"Bohemian Hotel Celebration, Autograph Collection",Hotel,700 Bloom St,at Celebration Pl,28.317021,-81.541176,"[{'label': 'display', 'lat': 28.3170214, 'lng'...",10710,34747,US,Celebration,FL,United States,"[700 Bloom St (at Celebration Pl), Celebration...",4b256951f964a520787124e3
8,The Palms Hotel and Villas,Hotel,3100 Parkway Blvd,NaN,28.341036,-81.536006,"[{'label': 'display', 'lat': 28.34103632634801...",8572,34747,US,Kissimmee,FL,United States,"[3100 Parkway Blvd, Kissimmee, FL 34747, Unite...",4bc33ad5dce4eee1a580719d
9,Red Lion Hotel Orlando Kissimmee Maingate,Hotel,7300 W Irlo Bronson Memorial Hwy,NaN,28.331963,-81.583320,"[{'label': 'display', 'lat': 28.33196337106309...",8214,34747,US,Kissimmee,FL,United States,"[7300 W Irlo Bronson Memorial Hwy, Kissimmee, ...",5697607e498e9e387a3196d0


In [18]:
FLid=[]
for index,row in Y.iterrows():
    try:
        H_id=row['id']
        uri='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(H_id,CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(uri).json()
        Hotel_id = result['response']['venue']['rating']
        FLid.append(Hotel_id)
    except:
        FLid.append('NaN')
 
print(FLid)


[9.2, 7.6, 9.0, 5.5, 7.3, 8.1, 8.1, 8.7, 6.8, 7.6, 5.1, 6.0, 7.2, 6.7, 'NaN', 6.6, 6.1, 'NaN']


In [19]:
Y['rating'] = FLid

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
Y['rating']= Y['rating'].astype(str)
dropvalues = Y[ Y['rating'] == 'NaN' ].index
Y.drop(dropvalues , inplace=True)
Y.reset_index(drop=True, inplace=True)
Y

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id,rating
0,Loews Portofino Bay Hotel at Universal Orlando,Hotel,5601 Universal Blvd,NaN,28.480846,-81.460843,"[{'label': 'display', 'lat': 28.48084591771423...",14623,32819,US,Orlando,FL,United States,"[5601 Universal Blvd, Orlando, FL 32819, Unite...",4b05868cf964a5206c6522e3,9.2
1,Rosen Centre Hotel,Hotel,9840 International Dr,NaN,28.424350,-81.464479,"[{'label': 'display', 'lat': 28.42434983796381...",11829,32819,US,Orlando,FL,United States,"[9840 International Dr, Orlando, FL 32819, Uni...",4b05868af964a520266522e3,7.6
2,"Grand Bohemian Hotel Orlando, Autograph Collec...",Hotel,325 South Orange Avenue,NaN,28.538690,-81.378645,"[{'label': 'display', 'lat': 28.5386903, 'lng'...",24911,32801,US,Orlando,FL,United States,"[325 South Orange Avenue, Orlando, FL 32801, U...",4b05868bf964a520546522e3,9.0
3,Seralago Hotel & Suites Main Gate East,Hotel,5678 W Irlo Bronson Memorial Hwy,Seralago Blvd,28.331714,-81.511398,"[{'label': 'display', 'lat': 28.3317138, 'lng'...",10850,34746,US,Kissimmee,FL,United States,[5678 W Irlo Bronson Memorial Hwy (Seralago Bl...,4b05868bf964a5205c6522e3,5.5
4,"Castle Hotel, Autograph Collection",Hotel,8629 International Drive,NaN,28.440175,-81.469128,"[{'label': 'display', 'lat': 28.4401754, 'lng'...",11828,32819,US,Orlando,FL,United States,"[8629 International Drive, Orlando, FL 32819, ...",5009edfde4b0b84fa25c768e,7.3
5,Melia Orlando Suite Hotel at Celebration,Hotel,225 Celebration Pl,NaN,28.331508,-81.537966,"[{'label': 'display', 'lat': 28.33150816213022...",9389,34747,US,Celebration,FL,United States,"[225 Celebration Pl, Celebration, FL 34747, Un...",54c11ca6498ee52a436a6d4f,8.1
6,Homerun Hotel Buildings,Hotel,All Star Sports Resort,Walt Disney World,28.341798,-81.574689,"[{'label': 'display', 'lat': 28.34179838953522...",7171,32830,US,Lake Buena Vista,FL,United States,"[All Star Sports Resort (Walt Disney World), L...",4ee155d330f87dc0a5df97bf,8.1
7,"Bohemian Hotel Celebration, Autograph Collection",Hotel,700 Bloom St,at Celebration Pl,28.317021,-81.541176,"[{'label': 'display', 'lat': 28.3170214, 'lng'...",10710,34747,US,Celebration,FL,United States,"[700 Bloom St (at Celebration Pl), Celebration...",4b256951f964a520787124e3,8.7
8,The Palms Hotel and Villas,Hotel,3100 Parkway Blvd,NaN,28.341036,-81.536006,"[{'label': 'display', 'lat': 28.34103632634801...",8572,34747,US,Kissimmee,FL,United States,"[3100 Parkway Blvd, Kissimmee, FL 34747, Unite...",4bc33ad5dce4eee1a580719d,6.8
9,Red Lion Hotel Orlando Kissimmee Maingate,Hotel,7300 W Irlo Bronson Memorial Hwy,NaN,28.331963,-81.583320,"[{'label': 'display', 'lat': 28.33196337106309...",8214,34747,US,Kissimmee,FL,United States,"[7300 W Irlo Bronson Memorial Hwy, Kissimmee, ...",5697607e498e9e387a3196d0,7.6


In [21]:
Y['rating']= Y['rating'].astype(float)
dropvalues2 = Y[ Y['rating'] < 6 ].index
Y.drop(dropvalues2 , inplace=True)
Y.reset_index(drop=True, inplace=True)
Y

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id,rating
0,Loews Portofino Bay Hotel at Universal Orlando,Hotel,5601 Universal Blvd,NaN,28.480846,-81.460843,"[{'label': 'display', 'lat': 28.48084591771423...",14623,32819,US,Orlando,FL,United States,"[5601 Universal Blvd, Orlando, FL 32819, Unite...",4b05868cf964a5206c6522e3,9.2
1,Rosen Centre Hotel,Hotel,9840 International Dr,NaN,28.424350,-81.464479,"[{'label': 'display', 'lat': 28.42434983796381...",11829,32819,US,Orlando,FL,United States,"[9840 International Dr, Orlando, FL 32819, Uni...",4b05868af964a520266522e3,7.6
2,"Grand Bohemian Hotel Orlando, Autograph Collec...",Hotel,325 South Orange Avenue,NaN,28.538690,-81.378645,"[{'label': 'display', 'lat': 28.5386903, 'lng'...",24911,32801,US,Orlando,FL,United States,"[325 South Orange Avenue, Orlando, FL 32801, U...",4b05868bf964a520546522e3,9.0
3,"Castle Hotel, Autograph Collection",Hotel,8629 International Drive,NaN,28.440175,-81.469128,"[{'label': 'display', 'lat': 28.4401754, 'lng'...",11828,32819,US,Orlando,FL,United States,"[8629 International Drive, Orlando, FL 32819, ...",5009edfde4b0b84fa25c768e,7.3
4,Melia Orlando Suite Hotel at Celebration,Hotel,225 Celebration Pl,NaN,28.331508,-81.537966,"[{'label': 'display', 'lat': 28.33150816213022...",9389,34747,US,Celebration,FL,United States,"[225 Celebration Pl, Celebration, FL 34747, Un...",54c11ca6498ee52a436a6d4f,8.1
5,Homerun Hotel Buildings,Hotel,All Star Sports Resort,Walt Disney World,28.341798,-81.574689,"[{'label': 'display', 'lat': 28.34179838953522...",7171,32830,US,Lake Buena Vista,FL,United States,"[All Star Sports Resort (Walt Disney World), L...",4ee155d330f87dc0a5df97bf,8.1
6,"Bohemian Hotel Celebration, Autograph Collection",Hotel,700 Bloom St,at Celebration Pl,28.317021,-81.541176,"[{'label': 'display', 'lat': 28.3170214, 'lng'...",10710,34747,US,Celebration,FL,United States,"[700 Bloom St (at Celebration Pl), Celebration...",4b256951f964a520787124e3,8.7
7,The Palms Hotel and Villas,Hotel,3100 Parkway Blvd,NaN,28.341036,-81.536006,"[{'label': 'display', 'lat': 28.34103632634801...",8572,34747,US,Kissimmee,FL,United States,"[3100 Parkway Blvd, Kissimmee, FL 34747, Unite...",4bc33ad5dce4eee1a580719d,6.8
8,Red Lion Hotel Orlando Kissimmee Maingate,Hotel,7300 W Irlo Bronson Memorial Hwy,NaN,28.331963,-81.583320,"[{'label': 'display', 'lat': 28.33196337106309...",8214,34747,US,Kissimmee,FL,United States,"[7300 W Irlo Bronson Memorial Hwy, Kissimmee, ...",5697607e498e9e387a3196d0,7.6
9,Best Western Orlando Convention Center Hotel,Hotel,6301 Westwood Blvd,NaN,28.419305,-81.464354,"[{'label': 'display', 'lat': 28.419305, 'lng':...",11756,32821,US,Orlando,FL,United States,"[6301 Westwood Blvd, Orlando, FL 32821, United...",4ead5492e5fad025f8129ccd,6.0


In [22]:
#Average distance between WaltDisney resort and all above rating 6 hotels
Dis_FL_hotel=Y['distance'].mean()
Dis_FL_hotel

11752.0

In [23]:
FL_map = folium.Map(location=[FL_latitude, FL_longitude], zoom_start=12) # generate map centred around the WaltDisney resort,Florida

# add a red circle marker to represent the WaltDisney resort,Florida
folium.features.CircleMarker(
    [FL_latitude, FL_longitude],
    radius=10,
    color='red',
    popup='WaltDisney resort,Florida',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(FL_map)

# add the Hotels as blue circle markers
for lat, lng, label in zip(Y.lat, Y.lng, Y.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(FL_map)

# display map
FL_map

# Hospitals around Florida & California

In [24]:
search_query = 'Hospitals'
#50000 = 50KM
radius = 50000
print(search_query + ' .... OK!')

Hospitals .... OK!


In [25]:
#Links to hotels around disney land california
url_CA_Hospital = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, CA_latitude, CA_longitude, VERSION, search_query, radius, LIMIT)
url_FL_Hospital = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, FL_latitude, FL_longitude, VERSION, search_query, radius, LIMIT)

# Hospitals near DisneyLand Resort ,California

In [26]:
#fetch result for hospitals in california
results_CA = requests.get(url_CA_Hospital).json()
Hospital_CA = results_CA['response']['venues']
# tranform venues into a dataframe CA
dataframe_CA2 = pd.json_normalize(Hospital_CA)
dataframe_CA2.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4d2611c2c0772c0f32bc9e88,Kindred Hospital Santa Ana,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1592678390,False,1901 College Ave,33.762282,-117.891513,"[{'label': 'display', 'lat': 33.76228205491646...",6285,92706,US,Santa Ana,CA,United States,"[1901 College Ave, Santa Ana, CA 92706, United...",79871563,NaN
1,59de4b1e67af3a10092fe3c9,Alta Hospitals System,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1592678390,False,12501 Imperial Hwy Ste 550,33.917692,-118.065820,"[{'label': 'display', 'lat': 33.917692, 'lng':...",17791,90650,US,Norwalk,CA,United States,"[12501 Imperial Hwy Ste 550 (Bloomfield Ave), ...",NaN,Bloomfield Ave
2,59de7ab8da7080280f576719,Alta Hospitals System,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1592678390,False,12501 Imperial Hwy Ste 550,33.917792,-118.065821,"[{'label': 'display', 'lat': 33.917792, 'lng':...",17798,90650,US,Norwalk,CA,United States,"[12501 Imperial Hwy Ste 550 (Bloomfield Ave), ...",NaN,Bloomfield Ave
3,4bb7689c941ad13aae9c20e3,Anaheim General Hospital Sub-acute,[],v-1592678390,False,NaN,33.876483,-117.997602,"[{'label': 'display', 'lat': 33.87648316004001...",10039,90621,US,Buena Park,CA,United States,"[Buena Park, CA 90621, United States]",NaN,NaN
4,4f10bd51e4b0d85e51677244,Kindred Hospital South Bay - Tri-City Campus,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1592678390,False,21530 Pioneer Blvd,33.833513,-118.080246,"[{'label': 'display', 'lat': 33.83351310949877...",15009,90716,US,Hawaiian Gardens,CA,United States,"[21530 Pioneer Blvd, Hawaiian Gardens, CA 9071...",79871573,NaN


In [27]:
#cleaning CA DATA
filtered_columns = ['name', 'categories'] + [col for col in dataframe_CA2.columns if col.startswith('location.')] + ['id']
dataframe_filtered_CA2 = dataframe_CA2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_CA2['categories'] = dataframe_filtered_CA2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_CA2.columns = [column.split('.')[-1] for column in dataframe_filtered_CA2.columns]

X_hos = dataframe_filtered_CA2[dataframe_filtered_CA2['categories']=='Hospital']
X_hos.reset_index(drop=True, inplace=True)
X_hos

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Kindred Hospital Santa Ana,Hospital,1901 College Ave,33.762282,-117.891513,"[{'label': 'display', 'lat': 33.76228205491646...",6285,92706,US,Santa Ana,CA,United States,"[1901 College Ave, Santa Ana, CA 92706, United...",NaN,4d2611c2c0772c0f32bc9e88
1,Kindred Hospital South Bay - Tri-City Campus,Hospital,21530 Pioneer Blvd,33.833513,-118.080246,"[{'label': 'display', 'lat': 33.83351310949877...",15009,90716,US,Hawaiian Gardens,CA,United States,"[21530 Pioneer Blvd, Hawaiian Gardens, CA 9071...",NaN,4f10bd51e4b0d85e51677244
2,HOAG Hospital Surgery Lounge,Hospital,NaN,33.624320,-117.929637,"[{'label': 'display', 'lat': 33.62431981156801...",21095,92663,US,Newport Beach,CA,United States,"[Newport Beach, CA 92663, United States]",NaN,4ecc947d8b81fbfbaa776412
3,Hospital São Camilo,Hospital,Marcelo Cândia,33.889957,-117.615318,"[{'label': 'display', 'lat': 33.88995742797851...",29395,92880,US,Macapá,CA,United States,"[Marcelo Cândia, Macapá, CA 92880, United States]",NaN,50dc983ce4b0ab7ccf64dba9
4,Vista Hospital San Gabriel Valley,Hospital,NaN,34.063024,-117.967510,"[{'label': 'display', 'lat': 34.063024, 'lng':...",28109,91706,US,Baldwin Park,CA,United States,"[Baldwin Park, CA 91706, United States]",NaN,4c0dba7dd64c0f47683b275d
5,Kindred Hospital - San Gabriel Valley,Hospital,845 North Lark Ellen Ave,34.086941,-117.917526,"[{'label': 'display', 'lat': 34.08694098653849...",30423,91791,US,West Covina,CA,United States,"[845 North Lark Ellen Ave (Badillo St), West C...",Badillo St,4bc4fb620969a593274e636e
6,Kindred Hospital South Bay,Hospital,1246 W 155th St,33.890110,-118.297360,"[{'label': 'display', 'lat': 33.89010974308726...",35935,90247,US,Gardena,CA,United States,"[1246 W 155th St, Gardena, CA 90247, United St...",NaN,4edfec70e5fa0be345eeaf52


In [28]:
Dis_CA_hospital=X_hos['distance'].mean()
Dis_CA_hospital

23750.14285714286

In [30]:
for lat, lng, label in zip(X_hos.lat, X_hos.lng, X_hos.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(CA_map)

# display map
CA_map

# Hospitals near WaltDisney Resort ,Florida

In [31]:
#fetch result for hospitals in Florida
results_FL = requests.get(url_FL_Hospital).json()
Hospital_FL = results_FL['response']['venues']
# tranform venues into a dataframe Fl
dataframe_FL2 = pd.json_normalize(Hospital_FL)
dataframe_FL2.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address
0,4bbf7feef8219c745d14b010,Clermont Animal Hospital South,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",v-1592678420,False,28.339120,-81.671371,"[{'label': 'display', 'lat': 28.33911976666666...",11367,34711,US,Clermont,FL,United States,"[Clermont, FL 34711, United States]",NaN
1,4f21b05ce4b028522e970a66,Florida Hospital Sports Medicine & Rehabilitation,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1592678420,False,28.328410,-81.540581,"[{'label': 'display', 'lat': 28.32841047058464...",9579,34747,US,Celebration,FL,United States,"[400 Celebration Pl, Celebration, FL 34747, Un...",400 Celebration Pl
2,4fcdf3fee4b06527ab2978fc,Florida Hospital Sports and Rehabilitation,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1592678420,False,28.522691,-81.470570,"[{'label': 'display', 'lat': 28.522691, 'lng':...",17071,32835,US,Orlando,FL,United States,"[Orlando, FL 32835, United States]",NaN
3,4cfce729c671721e88f9ca18,Florida Hospital Sports Medicine and Rehab,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1592678420,False,28.522716,-81.470460,"[{'label': 'display', 'lat': 28.522716, 'lng':...",17080,32835,US,Orlando,FL,United States,"[1603 S Hiawassee Rd, Orlando, FL 32835, Unite...",1603 S Hiawassee Rd
4,4dc3dd432271f2705103e11b,South Lake Hospital Surgery Center,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1592678420,False,28.553990,-81.723151,"[{'label': 'display', 'lat': 28.55398977222865...",21428,NaN,US,Clermont,FL,United States,"[Clermont, FL, United States]",NaN


In [32]:
#cleaning FL DATA
filtered_columns = ['name', 'categories'] + [col for col in dataframe_FL2.columns if col.startswith('location.')] + ['id']
dataframe_filtered_FL2 = dataframe_FL2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_FL2['categories'] = dataframe_filtered_FL2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_FL2.columns = [column.split('.')[-1] for column in dataframe_filtered_FL2.columns]

Y_hos = dataframe_filtered_FL2[dataframe_filtered_FL2['categories']=='Hospital']
Y_hos.reset_index(drop=True, inplace=True)
Y_hos

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,id
0,South Lake Hospital Surgery Center,Hospital,28.553990,-81.723151,"[{'label': 'display', 'lat': 28.55398977222865...",21428,NaN,US,Clermont,FL,United States,"[Clermont, FL, United States]",NaN,4dc3dd432271f2705103e11b
1,Southlake Hospital Second-Floor Surgical,Hospital,28.552380,-81.722130,"[{'label': 'display', 'lat': 28.55238, 'lng': ...",21226,NaN,US,Clermont,FL,United States,"[Clermont, FL, United States]",NaN,51504ba5e4b06ae7695d0d76
2,Florida hospital south,Hospital,28.571373,-81.377273,"[{'label': 'display', 'lat': 28.57137332571273...",27326,32804,US,Orlando,FL,United States,"[Orlando, FL 32804, United States]",NaN,4cc0cc54d363721e673356c9
3,Florida hospital surgical rapid in/out - room ...,Hospital,28.575871,-81.369324,"[{'label': 'display', 'lat': 28.57587051391601...",28238,32803,US,Orlando,FL,United States,"[Orlando, FL 32803, United States]",NaN,50f1a5a8e4b0f9cd366d9834
4,Florida Hospital South HBO,Hospital,28.573720,-81.374384,"[{'label': 'display', 'lat': 28.57372005068741...",27711,32804,US,Orlando,FL,United States,"[Orlando, FL 32804, United States]",NaN,4f4521e2e4b0c24ce33ebae1
5,Florida Hospital Stress Lab,Hospital,28.573462,-81.369635,"[{'label': 'display', 'lat': 28.5734623, 'lng'...",28037,32803,US,Orlando,FL,United States,"[601 E Rollins St, Orlando, FL 32803, United S...",601 E Rollins St,4e1affa4d4c0993513ea4ff3
6,Florida Hospital South (Labor And Delivery),Hospital,28.575377,-81.370341,"[{'label': 'display', 'lat': 28.57537652882011...",28127,32803,US,Orlando,FL,United States,"[601 E Rollins St, Orlando, FL 32803, United S...",601 E Rollins St,4d517204bd6ff04d75ddf10c
7,Florida Hospital South Outpatient Pediactric C...,Hospital,28.575015,-81.363742,"[{'label': 'display', 'lat': 28.57501484684325...",28583,32804,US,Orlando,FL,United States,"[2501 N Orange Ave, Orlando, FL 32804, United ...",2501 N Orange Ave,50b4d27be4b047c32bf9a2c8
8,Florida Hospital South Echo,Hospital,28.573894,-81.369389,"[{'label': 'display', 'lat': 28.573894, 'lng':...",28086,NaN,US,Orlando,FL,United States,"[Orlando, FL, United States]",NaN,4d6c140e3a6bf04d46765b7d
9,Florida Hospital Surgical Rapid In/Out,Hospital,28.576049,-81.369756,"[{'label': 'display', 'lat': 28.57604939148377...",28220,32803,US,NaN,Florida,United States,"[FL 32803, United States]",NaN,4ed63eb3e300d4f60435ef97


In [33]:
Dis_FL_hospital=Y_hos['distance'].mean()
Dis_FL_hospital

26698.2

In [34]:
for lat, lng, label in zip(Y_hos.lat, Y_hos.lng, Y_hos.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(FL_map)

# display map
FL_map

In [35]:
if (Dis_CA_hotel < Dis_FL_hotel) & (Dis_CA_hospital < Dis_FL_hospital) :
    print("Optimum choise is : DisneyLand Resort , California")
else:
    print("Optimum choise is : WaltDisney Resort , Florida")

Optimum choise is : DisneyLand Resort , California
